# Random forests intro and regression

Random Forests are powerful machine learning algorithms used for supervised classification and regression. Random forests works by averaging the predictions of the multiple and randomized decision trees. Decision trees tends to overfit and so by combining multiple decision trees, the effect of overfitting can be minimized. 

Random Forests are type of ensemble models. More about ensembles models in the next notebook. 

Different to other learning algorithms, random forests provide a way to find the importance of each feature and this is implemented in Sklearn. 

## Imports

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import pytest
import ipytest
import unittest

ipytest.autoconfig()

## Loading the data

In this regression task with random forests, we will use the Machine CPU (Central Processing Unit) dataset which is available at [OpenML](https://www.openml.org/t/5492).

If you are reading this, it's very likely that you know CPU or you have once(or many times) thought about it when you were buying your computer. In this notebook, we will predict the relative performance of the CPU given the following data: 

* MYCT: machine cycle time in nanoseconds (integer)
* MMIN: minimum main memory in kilobytes (integer)
* MMAX: maximum main memory in kilobytes (integer)
* CACH: cache memory in kilobytes (integer)
* CHMIN: minimum channels in units (integer)
* CHMAX: maximum channels in units (integer)
* PRP: published relative performance (integer) (target variable)

In [ ]:
# Let's hide warnings

import warnings

warnings.filterwarnings("ignore")

In [ ]:
machine_data = pd.read_csv("../../../assets/data/machine_cup.csv")

In [ ]:
type(machine_data)

In [ ]:
machine_data.shape

In [ ]:
machine_data.head()

## Tasks and roles

### Task 1: Exploratory analysis

Before doing exploratory analysis, let's get the training and test data. 

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(machine_data, test_size=0.2, random_state=20)
print(
    "The size of training data is: {} \nThe size of testing data is: {}".format(
        len(train_data), len(test_data)
    )
)

#### Part 1: The histogram

In [ ]:
def df_hist(df):
    if df is not None and not df.empty:
        df.hist(bins = 50, figsize = (15, 10))

In [ ]:
df_hist(train_data)
plt.show()

<h5><font color=blue>Check result by executing below... 📝</font></h5>

In [ ]:
%%ipytest -qq

from unittest.mock import Mock, patch

class TestDFHist(unittest.TestCase):
  
    def test_df_hist_happy_case(self):
        # assign
        test_df = Mock(return_value=pd.DataFrame(
            {
                'c1': [1, 2, 3, 4, 5], 
            }
        ))
        test_df.empty = False
        
        with patch.object(test_df, 'hist') as mock_df_hist:
            # act
            actual_result = df_hist(test_df)

            # assert
            mock_df_hist.assert_called_once()

    def test_df_hist_with_empty_df(self):
        # assign
        test_df = Mock(return_value=pd.DataFrame())
        
        with patch.object(test_df, 'hist') as mock_df_hist:
            # act
            actual_result = df_hist(test_df)

            # assert
            mock_df_hist.assert_not_called()
            
    def test_df_hist_with_none_df(self):
        # assign
        test_df = Mock(return_value=None)
        
        with patch.object(test_df, 'hist') as mock_df_hist:
            # act
            actual_result = df_hist(test_df)

            # assert
            mock_df_hist.assert_not_called()

<div class="alert alert-info">
    
<details><summary>👩‍💻 <b>Hint</b></summary>
    
You can consider to use <code>pandas.DataFrame.hist</code>.

</details>

</div>

#### Part 2: The pairplot

In [ ]:
def df_pairplot(df):
    if df is not None and not df.empty:
        sns.pairplot(df)

In [ ]:
df_pairplot(train_data)

<h5><font color=blue>Check result by executing below... 📝</font></h5>

In [ ]:
%%ipytest -qq

from unittest.mock import Mock, patch

class TestDFHist(unittest.TestCase):
  
    def test_df_pairplot_happy_case(self):
        # assign
        test_df = Mock(return_value=pd.DataFrame(
            {
                'c1': [1, 2, 3, 4, 5], 
                'c2': [2, 4, 6, 8, 10], 
            }
        ))
        test_df.empty = False
        
        with patch.object(sns, 'pairplot') as mock_pairplot:
            # act
            actual_result = df_pairplot(test_df)

            # assert
            mock_pairplot.assert_called_once_with(test_df)

    def test_df_pairplot_with_empty_df(self):
        # assign
        test_df = Mock(return_value=pd.DataFrame())
        
        with patch.object(sns, 'pairplot') as mock_df_pairplot:
            # act
            actual_result = df_pairplot(test_df)

            # assert
            mock_df_pairplot.assert_not_called()
            
    def test_df_pairplot_with_none_df(self):
        # assign
        test_df = Mock(return_value=None)
        
        with patch.object(sns, 'pairplot') as mock_df_pairplot:
            # act
            actual_result = df_pairplot(test_df)

            # assert
            mock_df_pairplot.assert_not_called()

<div class="alert alert-info">
    
<details><summary>👩‍💻 <b>Hint</b></summary>
    
You can consider to use <code>seaborn.pairplot</code>.

</details>

</div>

#### Part 3: Check the train data

In [ ]:
def df_desc(df):
    if df is None:
        raise Exception('df cannot be None.')
    return df.describe()

In [ ]:
# Let's check the summary stats
df_desc(train_data)

In [ ]:
def df_null(df):
    if df is None:
        raise Exception('df cannot be None.')
    return df.isnull().sum()

In [ ]:
# Let's check the missing values
df_null(train_data)

Great! We don't have any missing values. 

#### Part 4: Look the correlation

In [ ]:
def df_corr(df):
    if df is not None and not df.empty:
        corr = df.corr()
        return corr

In [ ]:
corr = df_corr(train_data)
corr["class"]

In [ ]:
def df_heat(correlation):
    if correlation is not None:
        return sns.heatmap(correlation, annot=True, cmap="crest")

In [ ]:
## Visualizing correlation

plt.figure(figsize=(12, 7))
df_heat(corr)

### Task 2: Data preprocessing

It is here that we prepare the data to be in the proper format for the machine learning model. 
Let's set up a pipeline to scale features but before that, let's take training input data and labels.

In [ ]:
X_train = train_data.drop("class", axis=1)
y_train = train_data["class"]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

scale_pipe = Pipeline([("scaler", StandardScaler())])
X_train_scaled = scale_pipe.fit_transform(X_train)

### Task 3: Training random forests regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(
    min_samples_split=2, bootstrap=False, random_state=42, n_jobs=-1
)

forest_reg.fit(X_train_scaled, y_train)



### Task 4: Evaluating random forests regressor

Let's first check the root mean squarred errr on the training. It is not advised to evaluate the model on the test data since we haven't improved it yet. we will make a function to make it easier and to avoid repetitions. 

In [ ]:
from sklearn.metrics import mean_squared_error


def predict(input_data, model, labels):
    """
    Take the input data, model and labels and return predictions

    """

    preds = model.predict(input_data)
    mse = mean_squared_error(labels, preds)
    rmse = np.sqrt(mse)

    return rmse

In [ ]:
predict(X_train_scaled, forest_reg, y_train)

### Task 5: Improving random forests

In [ ]:
forest_reg.get_params()

We will use GridSearch to find the best hyperparameters that we can use to retrain the model with. By setting the `refit` to `True`, the random forest will be automatically retrained on the dataset with the best hyperparameters. By default, `refit` is True.

This will take too long. 

In [ ]:
from sklearn.model_selection import GridSearchCV

params_grid = {
    "n_estimators": [100, 200, 300, 400, 500],
    "max_leaf_nodes": list(range(2, 52)),
}

# refit is true by default. The best estimator is trained on the whole dataset

grid_search = GridSearchCV(
    RandomForestRegressor(min_samples_split=2, bootstrap=False, random_state=42),
    params_grid,
    verbose=1,
    cv=5,
)

grid_search.fit(X_train_scaled, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
forest_best = grid_search.best_estimator_

Let's make prediction on the training data again

In [ ]:
predict(X_train_scaled, forest_best, y_train)

Surprisingly, by searching model hyperparameters, the model did not improve. Can you guess why? We can observe many things while running Grid Search and reading about the [random forests](https://scikit-learn.org/stable/modules/ensemble.html#forests-of-randomized-trees). If you can't get good results, set the `bootstrap` to False. It is true by default, and that means that you are training on samples of the training set instead of the whole training set. Try going back to the orginal model and change it to True and note how the prediction changes. Also learn more about the other hyperparameters. 

### Task 6: Feature importance 

Different to other machine learning models, random forests can show how each feature contributed to the model generalization. Let's find it.
The results are values between 0 and 1. The closer to 1, the good the feature was to the model.

In [ ]:
feat_import = forest_best.feature_importances_

feat_dict = {"Features": X_train.columns, "Feature Importance": feat_import}

pd.DataFrame(feat_dict)

As you can see above, the most 2 features which contributed to the prediction of the relative performance of the CPU are `MMAX` which is the Maximum Main Memory in Kilobytes and `CACH` (cache memory). 

It makes sense that the model was able to find that out. Main memory (RAM, Read Only Memory) and cache memory (which stores frequently used information thus facilitating faster processing and quick retrieval of information) are the two most factors of the CPU performance and if you are going to buy a new computer, you want to have high RAM and cache memory in order to have a powerful machine that can process/compute and retrieve things faster. 

### Task 7: Evaluating the Model on the Test Set

Let us evaluate the model on the test set. But we need first run the pipeline on the test data. Note that we only transform (not fit_transform).

In [ ]:
X_test = test_data.drop("class", axis=1)
y_test = test_data["class"]

test_scaled = scale_pipe.transform(X_test)

In [ ]:
predict(test_scaled, forest_best, y_test)

The results on the test set is not appealing, and it is a sign that the model is still overfitting the data(it is doing well on the training set and poor on the new data). One way to improve the model can be to regularize the model by searching more best hyperparameters and increasing the data and data quality. The later is what can improve the model in many scenarios. 

This is the end of this assignment. We have learned the fundamental idea behind the random forests, and used it to predict the CPU performance. In the next assignment, we will use it for classification task and we will use a real world dataset so that we can practically improve the random forests.

## Acknowledgments

Thanks to Nyandwi for creating the open-source course [Machine Learning complete](https://github.com/Nyandwi/machine_learning_complete). It inspires the majority of the content in this chapter.